In [54]:
import os
import re
import gc
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import warnings

warnings.simplefilter('ignore')

pd.set_option('max_columns', None)
pd.set_option('max_rows', 200)

In [55]:
train = pd.read_csv('G:/kaggle竞赛GM之路！！！/竞赛ing/2021CCF系统认证风险预测/train_dataset.csv', sep='\t')
test = pd.read_csv('G:/kaggle竞赛GM之路！！！/竞赛ing/2021CCF系统认证风险预测/test_dataset.csv', sep='\t')

In [50]:
data = pd.concat([train, test])

#剔除常量
data.drop(['client_type', 'browser_source'], axis=1, inplace=True)

#填充缺失值
data['auth_type'].fillna('na', inplace=True)


#拆分地址
data['location_first_lvl'] = data['location'].astype(str).apply(lambda x: json.loads(x)['first_lvl'])
data['location_sec_lvl'] = data['location'].astype(str).apply(lambda x: json.loads(x)['sec_lvl'])
data['location_third_lvl'] = data['location'].astype(str).apply(lambda x: json.loads(x)['third_lvl'])


#类别特征处理
for col in tqdm(['user_name', 'action', 'auth_type', 'ip',
				'ip_location_type_keyword', 'ip_risk_level', 'location', 'device_model',
				'os_type', 'os_version', 'browser_type', 'browser_version',
				'bus_system_code', 'op_target', 'location_first_lvl', 'location_sec_lvl',
				'location_third_lvl']):
	lbl = LabelEncoder()
	data[col] = lbl.fit_transform(data[col])


# 核心特征挖掘
# 异常时间，短时高频，地址异常，设备异常

data['op_date'] = pd.to_datetime(data['op_date'])

# 时间差
data['op_ts'] = data['op_date'].values.astype(np.int64)
data = data.sort_values(by=['user_name', 'op_ts']).reset_index(drop=True)
data['last_ts'] = data.groupby(['user_name'])['op_ts'].shift(1)
data['ts_diff1'] = data['op_ts'] - data['last_ts']


# 是否超过23点
data['hour'] = data['op_date'].dt.hour
data['past_23'] = list(map(lambda x: 1 if x>=23 else 0,data['hour']))

# 当天第几次登录
#data['date'] = data['op_date'].dt.date
#data['log_times'] = data.groupby(['user_name','date'])['op_ts'].rank()



# 相对于常规操作的偏离





# 剔除多余特征

data.drop(['location','op_ts','last_ts','hour'], axis=1, inplace=True)

  0%|          | 0/17 [00:00<?, ?it/s]

In [51]:
train = data[data['risk_label'].notna()]
test = data[data['risk_label'].isna()]


ycol = 'risk_label'
feature_names = list(filter(lambda x: x not in [ycol, 'session_id', 'op_date'], train.columns))

model = lgb.LGBMClassifier(objective='binary',
							boosting_type='gbdt',
							tree_learner='serial',
							num_leaves=2 ** 8,
							max_depth=16,
							learning_rate=0.2,
							n_estimators=10000,
							subsample=0.75,
							feature_fraction=0.55,
							reg_alpha=0.2,
							reg_lambda=0.2,
							random_state=0,
							is_unbalance=True,
							# scale_pos_weight=130,
							metric='auc')

oof = []
prediction = test[['session_id']]
prediction[ycol] = 0
df_importance_list = []

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(train[feature_names], train[ycol])):
	X_train = train.iloc[trn_idx][feature_names]
	Y_train = train.iloc[trn_idx][ycol]
	
	X_val = train.iloc[val_idx][feature_names]
	Y_val = train.iloc[val_idx][ycol]
	
	print('\nFold_{} Training ================================\n'.format(fold_id + 1))
	
	lgb_model = model.fit(X_train,
						Y_train,
						eval_names=['train', 'valid'],
						eval_set=[(X_train, Y_train), (X_val, Y_val)],
						verbose=500,
						eval_metric='auc',
						early_stopping_rounds=50)
	
	pred_val = lgb_model.predict_proba(X_val, num_iteration=lgb_model.best_iteration_)
	
	df_oof = train.iloc[val_idx][['session_id', ycol]].copy()
	df_oof['pred'] = pred_val[:, 1]
	oof.append(df_oof)
	
	pred_test = lgb_model.predict_proba(test[feature_names], num_iteration=lgb_model.best_iteration_)
	
	prediction[ycol] += pred_test[:, 1] / kfold.n_splits
	
	df_importance = pd.DataFrame({
		'column': feature_names,
		'importance': lgb_model.feature_importances_,
	})
	df_importance_list.append(df_importance)
	
	del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
	gc.collect()


Fold_1 Training ================================

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	train's auc: 0.528201	valid's auc: 0.49545

Fold_2 Training ================================

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[13]	train's auc: 0.729367	valid's auc: 0.543249

Fold_3 Training ================================

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[45]	train's auc: 0.790676	valid's auc: 0.489858

Fold_4 Training ================================

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[2]	train's auc: 0.551989	valid's auc: 0.568557

Fold_5 Training ================================

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[42]	train's auc: 0.787009	valid's auc: 0.596759

Fold_6 Training ======

In [52]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg('mean').sort_values(ascending=False).reset_index()
df_importance

column  importance
0                   ts_diff1     1084.40
1            bus_system_code      611.45
2                  user_name      550.15
3                  auth_type      400.75
4               browser_type      255.95
5                  op_target      209.30
6            browser_version      187.80
7                         ip      149.95
8         location_third_lvl       81.90
9               device_model       57.75
10                    action       56.90
11  ip_location_type_keyword       26.60
12                os_version       23.15
13                   past_23       16.60
14          location_sec_lvl       15.55
15             ip_risk_level       15.55
16        location_first_lvl       10.10
17                   os_type        0.95

In [53]:
df_oof = pd.concat(oof)
print('roc_auc_score', roc_auc_score(df_oof[ycol], df_oof['pred']))

roc_auc_score 0.5181790271590583


In [22]:
prediction['id'] = range(len(prediction))
prediction['id'] = prediction['id'] + 1
prediction = prediction[['id', 'risk_label']].copy()
prediction.columns = ['id', 'ret']
prediction.head()

id       ret
6147   1  0.207401
6148   2  0.444177
6149   3  0.446750
6150   4  0.494633
6151   5  0.477273

In [23]:
prediction.to_csv('G:/kaggle竞赛GM之路！！！/竞赛ing/2021CCF系统认证风险预测/submit_4.1/submission.csv', index=False)